<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/3-R%C3%A9seau%20de%20neurones%20en%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Réseaux de Neurones avec PyTorch

Les réseaux d'apprentissage profond ont tendance à être massifs avec des dizaines ou des centaines de couches, c'est de là que vient le terme « profond ». Vous pouvez construire l'un de ces réseaux profonds en utilisant uniquement des matrices de pondération comme nous l'avons fait dans le notebook précédent, mais en général, c'est très lourd et difficile à mettre en œuvre. PyTorch dispose d'un module intéressant, `nn`, qui fournit un moyen pratique de construire efficacement de grands réseaux de neurones.

In [ ]:
# Import necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

import matplotlib.pyplot as plt

Nous allons maintenant construire un réseau plus important qui peut résoudre un problème (auparavant) difficile, à savoir identifier du texte dans une image. Ici, nous utiliserons l'ensemble de données MNIST qui se compose de chiffres manuscrits en niveaux de gris. Chaque image mesure 28x28 pixels, vous pouvez voir un échantillon ci-dessous

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/mnist.png?raw=1'>

Notre objectif est de construire un réseau de neurones capable de prendre l'une de ces images et de prédire le chiffre affiché.

Tout d'abord, nous devons obtenir notre ensemble de données. Il est fourni via le package `torchvision`. Le code ci-dessous téléchargera l'ensemble de données MNIST, puis créera des ensembles de données d'entraînement et de test pour nous.

Probablement il y aura des messages d'erreur car le serveur de stockage a une protection CloudFlare. Cependant, une fois executé, le dataset doit être disponible (vous le verrez dans les paragraphes suivants).

In [ ]:
# The MNIST datasets are hosted on yann.lecun.com that has moved under CloudFlare protection
# Run this script to enable the datasets download
# Reference: https://github.com/pytorch/vision/issues/1938

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Nous avons les données d'entraînement chargées dans `trainloader` et nous en faisons un itérateur avec `iter(trainloader)`. Plus tard, nous l'utiliserons pour parcourir l'ensemble de données pour l'entraînement, comme

```python
for image, label in trainloader:
## do things with images and labels
```

Vous remarquerez que nous avons créé le `trainloader` avec une taille de *batch* de 64 et `shuffle=True`. La taille du batch est le nombre d'images que nous obtenons en une itération à partir du chargeur de données et que nous transmettons via notre réseau, souvent appelé un *batch*. Et `shuffle=True` lui indique de mélanger l'ensemble de données chaque fois que nous recommençons à parcourir le dataloader. Mais ici, nous récuperons simplement le premier batch afin que nous puissions vérifier les données. Nous pouvons voir ci-dessous que `images` n'est qu'un tensor de taille `(64, 1, 28, 28)`. Donc, 64 images par batch, 1 canal de couleur et 28x28 pixels.

In [ ]:
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(type(images))
print(images.shape)
print(labels.shape)

Voici ce que cette image ressemble :

In [ ]:
plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r');

Commençons par construire un réseau simple pour cet ensemble de données en utilisant des matrices poids et des multiplications de matrices. Ensuite, nous verrons comment le faire en utilisant le module `nn` de PyTorch qui fournit une méthode beaucoup plus pratique et puissante pour définir des architectures de réseau.

Les réseaux que vous avez vus jusqu'à présent sont appelés réseaux *entièrement connectés* ou *denses*. Chaque unité d'une couche est connectée à chaque unité de la couche suivante. Dans les réseaux entièrement connectés, l'entrée de chaque couche doit être un vecteur unidimensionnel (qui peut être empilé dans un tenseur 2D sous forme de *batches* de plusieurs exemples). Cependant, nos images sont des tensors 2D 28x28, nous devons donc les convertir en vecteurs 1D. En pensant aux tailles, nous devons convertir le batch d'images de forme `(64, 1, 28, 28)` en un batch d'images de forme `(64, 784)`, 784 étant 28 fois 28. Ceci est généralement appelé *aplatissement*, nous devons donc aplatir les images 2D en vecteurs 1D.

Auparavant, vous avez construit un réseau avec une unité (neurone) de sortie. Ici, nous avons besoin de 10 unités de sortie, une pour chaque chiffre. Nous voulons que notre réseau prédise le chiffre affiché dans une image, il faudra donc calculer les probabilités que l'image soit d'un chiffre ou d'une classe quelconque. Cela finit par être une distribution de probabilité discrète sur les classes (chiffres) qui nous indique la classe la plus probable pour l'image. Cela signifie que nous avons besoin de 10 unités de sortie pour les 10 classes (chiffres). Nous verrons ensuite comment convertir la sortie du réseau en une distribution de probabilité.

> **Exercice :** Aplatir le batch d'images `images`. Créez ensuite un réseau multicouche avec 784 unités d'entrée, 256 unités cachées et 10 unités de sortie en utilisant des tensors aléatoires pour les poids et les biais. Pour l'instant, utilisez une activation sigmoïde pour la couche cachée. Laissez la couche de sortie sans activation, nous en ajouterons une qui nous donnera une distribution de probabilité ensuite.

In [ ]:
## Votre solution


out = # la sortie du réseau doit avoir la forme (64,10)

Nous avons maintenant 10 sorties pour notre réseau. Nous voulons transmettre une image à notre réseau et obtenir une distribution de probabilité sur les classes qui nous indique à quelle(s) classe(s) l'image appartient probablement. Quelque chose qui ressemble à ceci :
<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/image_distribution.png?raw=1' width=500px>

Ici, nous voyons que la probabilité pour chaque classe est à peu près la même. Cela représente un réseau non formé, il n'a pas encore vu de données, il renvoie donc simplement une distribution uniforme avec des probabilités égales pour chaque classe.

Pour calculer cette distribution de probabilité, nous utilisons souvent la [**fonction softmax**](https://en.wikipedia.org/wiki/Softmax_function). Mathématiquement, cela ressemble à

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

Ce que cela fait, c'est écraser chaque entrée $x_i$ entre 0 et 1 et normaliser les valeurs pour vous donner une distribution de probabilité appropriée où les probabilités s'additionnent à un.

> **Exercice :** Implémentez une fonction `softmax` qui effectue le calcul softmax et renvoie les distributions de probabilité pour chaque exemple du batch. Notez que vous devrez faire attention aux formes lorsque vous faites cela. Si vous avez un tenseur `a` de forme `(64, 10)` et un tenseur `b` de forme `(64,)`, faire `a/b` vous donnera une erreur car PyTorch essaiera de faire la division sur les colonnes (appelée diffusion) mais vous obtiendrez une incompatibilité de taille.

La façon de voir les choses est la suivante : pour chacun des 64 exemples, vous ne voulez diviser que par une valeur, la somme du dénominateur. Vous avez donc besoin que `b` ait une forme de `(64, 1)`. De cette façon, PyTorch divisera les 10 valeurs de chaque ligne de `a` par la valeur de chaque ligne de `b`. Faites également attention à la façon dont vous prenez la somme. Vous devrez définir le mot-clé `dim` dans `torch.sum`. Le réglage `dim=0` prend la somme sur les lignes tandis que `dim=1` prend la somme sur les colonnes.

In [ ]:
def softmax(x):
    ## TODO: implémenter la fonction softmax ici

# Here, out should be the output of the network in the previous excercise with shape (64,10)
probabilities = softmax(out)

# Does it have the right shape? Should be (64, 10)
print(probabilities.shape)
# Does it sum to 1?
print(probabilities.sum(dim=1))

## Création de réseaux avec PyTorch

PyTorch fournit un module `nn` qui simplifie considérablement la création de réseaux. Voyons comment créer le même module que ci-dessus avec 784 entrées, 256 unités cachées, 10 unités de sortie et une sortie softmax.

In [ ]:
from torch import nn

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)

        # Define sigmoid activation and softmax output
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)

        return x

Passons en revue cela petit à petit.

```python
class Network(nn.Module):
```

Ici, nous héritons de `nn.Module`. Combiné avec `super().__init__()`, cela crée une classe qui suit l'architecture et fournit de nombreuses méthodes et attributs utiles. Il est obligatoire d'hériter de `nn.Module` lorsque vous créez une classe pour votre réseau. Le nom de la classe elle-même peut être n'importe quoi.

```python
self.hidden = nn.Linear(784, 256)
```

Cette ligne crée un module pour une transformation linéaire, $x\mathbf{W} + b$, avec 784 entrées et 256 sorties et l'affecte à `self.hidden`. Le module crée automatiquement les tensors de poids et de biais que nous utiliserons dans la méthode `forward`. Vous pouvez accéder aux tensors de poids et de biais une fois que le réseau (`net`) est créé avec `net.hidden.weight` et `net.hidden.bias`.

```python
self.output = nn.Linear(256, 10)
```

De même, cela crée une autre transformation linéaire avec 256 entrées et 10 sorties.

```python
self.sigmoid = nn.Sigmoid()
self.softmax = nn.Softmax(dim=1)
```

Ici, on a définit des opérations pour l'activation sigmoïde et la sortie softmax. La définition de `dim=1` dans `nn.Softmax(dim=1)` calcule le softmax sur les colonnes.

```python
def forward(self, x):
```

Les réseaux PyTorch créés avec `nn.Module` doivent avoir une méthode `forward` définie. Il prend un tensor `x` et le transmet aux opérations que vous avez définies dans la méthode `__init__`.

```python
x = self.hidden(x)
x = self.sigmoid(x)
x = self.output(x)
x = self.softmax(x)
```

Ici, le tensor d'entrée `x` est transmis à chaque opération et réaffecté à `x`. Nous pouvons voir que le tensor d'entrée passe par la couche cachée, puis une fonction sigmoïde, puis la couche de sortie et enfin la fonction softmax. Peu importe le nom que vous donnez aux variables ici, tant que les entrées et les sorties des opérations correspondent à l'architecture réseau que vous souhaitez créer. L'ordre dans lequel vous définissez les éléments dans la méthode `__init__` n'a pas d'importance, mais vous devrez séquencer correctement les opérations dans la méthode `forward`.

Nous pouvons maintenant créer un objet `Network`.

In [ ]:
# Create the network and look at it's text representation
model = Network()
model

You can define the network somewhat more concisely and clearly using the `torch.nn.functional` module. This is the most common way you'll see networks defined as many operations are simple element-wise functions. We normally import this module as `F`, `import torch.nn.functional as F`.

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        # Hidden layer with sigmoid activation
        x = F.sigmoid(self.hidden(x))
        # Output layer with softmax activation
        x = F.softmax(self.output(x), dim=1)

        return x

### Fonctions d'activation

Jusqu'ici, nous n'avons étudié que la fonction d'activation sigmoïde, mais en général, n'importe quelle fonction peut être utilisée comme fonction d'activation. La seule exigence est que pour qu'un réseau puisse approximer une fonction non linéaire, les fonctions d'activation doivent être non linéaires. Voici quelques exemples supplémentaires de fonctions d'activation courantes : Tanh (tangente hyperbolique) et ReLU (unité linéaire rectifiée).

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/activation.png?raw=1" width=700px>

En pratique, la fonction ReLU est la plus utilisée comme fonction d'activation pour les couches cachées.

### À vous de construire un réseau

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/mlp_mnist.png?raw=1" width=600px>

> **Exercice :** Créez un réseau avec 784 unités d'entrée, une couche cachée avec 128 unités et une activation ReLU, puis une couche cachée avec 64 unités et une activation ReLU, et enfin une couche de sortie avec une activation softmax comme indiqué ci-dessus. Vous pouvez utiliser une activation ReLU avec le module `nn.ReLU` ou la fonction `F.relu`.

Il est recommandé de nommer vos couches par leur type de réseau, par exemple 'fc' pour représenter une couche entièrement connectée. Lorsque vous codez votre solution, utilisez `fc1`, `fc2` et `fc3` comme noms de vos couches.

In [ ]:
## Votre solution ici


### Initialisation des poids et des biais

Les poids et autres sont automatiquement initialisés pour vous, mais il est possible de personnaliser la manière dont ils sont initialisés. Les poids et les biais sont des tensors attachés à la couche que vous avez définie, vous pouvez les obtenir avec `model.fc1.weight` par exemple.

In [ ]:
print(model.fc1.weight)
print(model.fc1.bias)

Pour une initialisation personnalisée, nous souhaitons modifier ces tensors en place. Ce sont en fait des *variables* autograd, nous devons donc récupérer les tensors réels avec `model.fc1.weight.data`. Une fois que nous avons les tensors, nous pouvons les remplir avec des zéros (pour les biais) ou des valeurs normales aléatoires.

In [ ]:
# Set biases to all zeros
model.fc1.bias.data.fill_(0)

In [ ]:
# sample from random normal with standard dev = 0.01
model.fc1.weight.data.normal_(std=0.01)

### Forward pass

Maintenant que nous avons un réseau, voyons ce qui se passe lorsque nous transmettons une image.

In [ ]:
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

# Grab some data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Resize images into a 1D vector, new shape is (batch size, color channels, image pixels)
images.resize_(64, 1, 784)
# or images.resize_(images.shape[0], 1, 784) to automatically get batch size

# Forward pass through the network
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
view_classify(img.view(1, 28, 28), ps)

Comme vous pouvez le voir ci-dessus, notre réseau n'a pratiquement aucune idée de ce qu'est ce chiffre. C'est parce que nous ne l'avons pas encore formé, tous les poids sont aléatoires !

### Utilisation de `nn.Sequential`

PyTorch fournit un moyen pratique de créer des réseaux comme celui-ci où un tenseur est transmis séquentiellement via des opérations, `nn.Sequential` ([documentation](https://pytorch.org/docs/master/nn.html#torch.nn.Sequential)). En utilisant ceci pour créer le réseau équivalent :

In [ ]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Forward pass through the network and display output
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])
helper.view_classify(images[0].view(1, 28, 28), ps)

Ici, notre modèle est le même que précédemment : 784 unités d'entrée, une couche cachée avec 128 unités, l'activation ReLU, une couche cachée de 64 unités, une autre ReLU, puis la couche de sortie avec 10 unités et la sortie softmax.

Les opérations sont disponibles en passant l'index approprié. Par exemple, si vous souhaitez obtenir la première opération linéaire et examiner les poids, vous utiliserez `model[0]`.

In [ ]:
print(model[0])
model[0].weight

Vous pouvez également transmettre un `OrderedDict` pour nommer les couches et les opérations individuelles, au lieu d'utiliser des entiers incrémentiels. Notez que les clés du dictionnaire doivent être uniques, donc _chaque opération doit avoir un nom différent_.

In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model

on peut ainsi accéder à chaque couche selon leurs noms

In [ ]:
print(model[0])
print(model.fc1)

Dans le prochain notebook, nous verrons comment nous pouvons entraîner un réseau neuronal à prédire avec précision les nombres apparaissant dans les images MNIST.